## О первичных преобразованиях датасета

Нужно загрузить датасет в DataLens.
Проблема -- датасет весит 1 ГБ, а загружать можно не более 10 файлов не более 200 МБ каждый.
200 МБ * 10 приблизительно равно 2 ГБ, так что данные совершенно точно поместятся в DataLens -- но не напрямую.

Идея -- уменьшить объём файла с данными, заменив строки числами. То, что получится, разбить по столбцам так, чтобы каждый файл не превышал 200 МБ, и тремя файлами загрузить в DataLens.

In [1]:
import pandas as pd

In [2]:
full = pd.read_csv('full.csv')
full.drop(full.filter(regex="Unname"),axis=1, inplace=True)

In [3]:
full

,userid,sessionid,from,timestamp,action,value,category,age,gender,city
0,user_10000,1,По рекламе,2021-12-29T08:52:55,mainpage,0.000000,None,24,Ж,Волгоград
1,user_10000,2,None,2022-01-03T08:53:04,category,0.000000,Красота и здоровье,24,Ж,Волгоград
2,user_10000,2,Из мессенджеров,2022-01-03T08:53:18,cart,24498.611274,Одежда и обувь,24,Ж,Волгоград
3,user_10000,2,None,2022-01-03T08:53:39,checkout,13800.000000,None,24,Ж,Волгоград
4,user_10000,2,None,2022-01-03T08:54:03,confirmation,13800.000000,None,24,Ж,Волгоград
...,...,...,...,...,...,...,...,...,...,...
7598784,user_12499,590,None,2025-01-03T02:01:24,search,0.000000,Товары для детей,51,Ж,Москва
7598785,user_12499,590,По рекламе,2025-01-03T02:01:30,search,0.000000,Продовольственные товары,51,Ж,Москва
7598786,user_12499,590,None,2025-01-03T02:01:50,product,0.000000,Продовольственные товары,51,Ж,Москва
7598787,user_12499,590,С сохраненных страниц,2025-01-03T02:01:52,cart,16123.586449,Продовольственные товары,51,Ж,Москва


### Заменим строки числами для уменьшения объёма

In [4]:
for i, name in enumerate(full['from'].unique()):
    print(i, name)
    full.loc[full['from'] == name, 'from'] = i

0 По рекламе
1 None
2 Из мессенджеров
3 Внутренние переходы
4 По ссылкам на сайтах
5 С почтовых рассылок
6 Из поисковых систем
7 С сохраненных страниц
8 Из рекомендательных систем
9 Прямые заходы


In [5]:
for i, name in enumerate(full['action'].unique()):
    print(i, name)
    full.loc[full['action'] == name, 'action'] = i

0 mainpage
1 category
2 cart
3 checkout
4 confirmation
5 search
6 product


In [6]:
for i, name in enumerate(full['category'].unique()):
    print(i, name)
    full.loc[full['category'] == name, 'category'] = i

0 None
1 Красота и здоровье
2 Одежда и обувь
3 Авто/мото товары
4 Товары для детей
5 Продовольственные товары
6 Электроника и бытовая техника


In [7]:
for i, name in enumerate(full['city'].unique()):
    print(i, name)
    full.loc[full['city'] == name, 'city'] = i

0 Волгоград
1 Санкт-Петербург
2 Москва
3 Нижний Новгород
4 Челябинск
5 Новосибирск
6 Омск
7 Екатеринбург
8 Уфа
9 Самара
10 Казань
11 Красноярск
12 Ростов-на-Дону
13 Воронеж


### Найдём и отметим первые и последние действия пользователей
(должно было быть полезным для дальнейших расчётов, но пригодилось не целиком, потому что особенности данных)

In [8]:
users_min = full.groupby('userid')['timestamp'].min()
full = full.merge(users_min, on='userid')
full['is_first'] = full['timestamp_x'] == full['timestamp_y']
full.drop(columns=['timestamp_y'], inplace=True)
full.rename(columns={"timestamp_x": "timestamp"}, inplace=True)

In [9]:
users_max = full.groupby('userid')['timestamp'].max()
full = full.merge(users_max, on='userid')
full['is_last'] = full['timestamp_x'] == full['timestamp_y']
full.drop(columns=['timestamp_y'], inplace=True)
full.rename(columns={"timestamp_x": "timestamp"}, inplace=True)

### Ещё немного изменений формата записей для уменьшения объёма данных

Проверяем, что все userid начинаются на user_, и убираем эту часть:

In [10]:
a = full['userid'].str.startswith('user_')
a.unique()

array([ True])

In [11]:
full['userid'] = full['userid'].str.slice(start=5)
full['userid'] = full['userid'].astype(int)

Дату и время из строки преобразуем в число-timestamp:

In [12]:
full['timestamp'] = pd.to_datetime(full['timestamp'])
full['timestamp'] = full['timestamp'].view(int) // 10**9

Округлим все value до сотых, потому что денежные единицы меньше копейки никого не интересуют:

In [13]:
full['value'] = full['value'].round(2)

Можно даже bool'ы к int'ам привести:

In [14]:
full['is_first'] = full['is_first'].astype(int)
full['is_last'] = full['is_last'].astype(int)

### Методом проб и ошибок разбиваем данные на три части по столбцам так, чтобы каждый файл весил менее 200 МБ

In [15]:
full.to_csv('part0.csv',
           columns=['userid', 'sessionid', 'from', 'category', 'action'])

In [ ]:
full.to_csv('part1.csv',
           columns=['timestamp', 'value'])

In [ ]:
full.to_csv('part2.csv',
           columns=['age', 'gender', 'city', 'is_first', 'is_last'])

Сработало!